# Amazon Bedrock AgentCore 런타임(Runtime)에서 A2A 에이전트(Agent) 호스팅 - AWS IAM 인바운드 인증(Authentication)

## 개요

이 튜토리얼에서는 AWS IAM을 인바운드 인증(Authentication)에 사용하여 Amazon Bedrock AgentCore 런타임(Runtime)에서 A2A(Agent-to-Agent) 에이전트(Agent)를 호스팅하는 방법을 배웁니다.

[A2A 프로토콜](https://a2a-protocol.org/dev/specification/)은 독립적인 AI 에이전트(Agent) 시스템 간의 통신을 촉진하기 위해 설계된 개방형 표준입니다. A2A는 전통적으로 인증(Authentication)에 OAuth/JWT 토큰을 사용하지만, AgentCore 런타임(Runtime)은 인바운드 요청에 AWS IAM 자격 증명을 구성(Configuration)할 수 있어 엔터프라이즈 보안 요구사항을 해결합니다.

### 튜토리얼 세부사항

| 정보                | 세부사항                                                  |
|:--------------------|:---------------------------------------------------------|
| 튜토리얼 유형       | 에이전트(Agent) 호스팅                                    |
| 프로토콜            | A2A (Agent-to-Agent)                                      |
| 인증(Authentication) | AWS IAM (SigV4)                                          |
| 튜토리얼 구성 요소   | IAM 인증(Authentication)으로 AgentCore 런타임(Runtime)에 A2A 에이전트(Agent) 호스팅 |
| 튜토리얼 분야       | 범용                                                      |
| 예제 복잡도         | 중급                                                      |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 Strands Agents     |

### 튜토리얼 아키텍처

이 튜토리얼에서는 IAM 인증(Authentication)으로 AgentCore 런타임(Runtime)에 A2A 에이전트(Agent)를 배포(Deployment)합니다.

데모 목적으로 `greet_user`와 `get_agent_info` 두 가지 도구(Tool)가 있는 간단한 에이전트(Agent)를 사용합니다.

### 튜토리얼 주요 기능

* Strands 프레임워크(Framework)로 A2A 에이전트(Agent) 생성
* 로컬에서 A2A 에이전트(Agent) 테스트
* Amazon Bedrock AgentCore 런타임(Runtime)에 A2A 에이전트(Agent) 호스팅
* IAM 인증(Authentication)(SigV4)으로 배포(Deployment)된 A2A 에이전트(Agent) 호출(Invocation)

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명 구성(Configuration)
* Amazon Bedrock AgentCore SDK
* Strands Agents 프레임워크(Framework)
* Docker 데몬 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os

In [ ]:
boto_session = Session()
region = boto_session.region_name

agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)

agent_name = "a2a_agent_iam"

## Understanding A2A (Agent-to-Agent Protocol)

A2A is a protocol that allows AI agents to communicate with each other. Key concepts:

* **Agent Card**: Metadata describing the agent's capabilities
* **Message Exchange**: Structured communication between agents
* **Tools**: Functions that agents can expose to other agents
* **IAM Authentication**: Using AWS SigV4 for secure authentication

AgentCore Runtime expects A2A agents to be hosted on `0.0.0.0:9000/` as the default path.

### Project Structure

```
agentcore-a2a-iam-sample/
├── agent.py              # Main A2A agent code
├── client.py             # Client for testing with IAM auth
├── requirements.txt      # Dependencies
└── hosting_a2a_iam_auth.ipynb  # This notebook
```

## Review Agent Code

Let's review the agent code that we'll deploy:

In [ ]:
!cat agent.py

### What This Code Does

* **Strands Agent**: Creates an agent using the Strands framework
* **@tool**: Decorator that turns Python functions into agent tools
* **A2AServer**: Wraps the agent to support A2A protocol
* **FastAPI**: Provides HTTP server for the agent
* **Tools**: Two simple tools demonstrating agent capabilities

## A2A(Agent-to-Agent 프로토콜) 이해

A2A는 AI 에이전트(Agent)들이 서로 통신할 수 있게 해주는 프로토콜입니다. 주요 개념:

* **에이전트(Agent) 카드**: 에이전트(Agent)의 기능을 설명하는 메타데이터
* **메시지 교환**: 에이전트(Agent) 간의 구조화된 통신
* **도구(Tool)**: 에이전트(Agent)가 다른 에이전트(Agent)에게 노출할 수 있는 함수
* **IAM 인증(Authentication)**: 안전한 인증(Authentication)을 위해 AWS SigV4 사용

AgentCore 런타임(Runtime)은 A2A 에이전트(Agent)가 기본 경로로 `0.0.0.0:9000/`에서 호스팅될 것으로 예상합니다.

### 프로젝트 구조

```
agentcore-a2a-iam-sample/
├── agent.py              # 메인 A2A 에이전트(Agent) 코드
├── client.py             # IAM 인증(Authentication)으로 테스트하기 위한 클라이언트
├── requirements.txt      # 종속성
└── hosting_a2a_iam_auth.ipynb  # 이 노트북
```

## 에이전트(Agent) 코드 검토

배포(Deployment)할 에이전트(Agent) 코드를 검토해 보겠습니다:

In [ ]:
print(f"Using AWS region: {region}")

required_files = ["agent.py", "requirements.txt"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="A2A",
    agent_name=agent_name,
)
print("Configuration completed ✓")

## Launching A2A Agent to AgentCore Runtime

Now that we have a Dockerfile, let's launch the A2A agent to the AgentCore Runtime. This will:

1. Create the Amazon ECR repository
2. Build and push the Docker image
3. Create the AgentCore Runtime
4. Deploy the agent

This may take several minutes...

In [ ]:
print("Launching A2A agent to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### 이 코드가 하는 일

* **Strands Agent**: Strands 프레임워크(Framework)를 사용하여 에이전트(Agent) 생성
* **@tool**: Python 함수를 에이전트(Agent) 도구(Tool)로 변환하는 데코레이터
* **A2AServer**: A2A 프로토콜을 지원하도록 에이전트(Agent)를 래핑
* **FastAPI**: 에이전트(Agent)를 위한 HTTP 서버 제공
* **도구(Tool)**: 에이전트(Agent) 기능을 시연하는 두 가지 간단한 도구(Tool)

In [ ]:
import os
os.environ['AGENT_ARN'] = launch_result.agent_arn

from client import *

# Test with a custom message
custom_message = "Please greet me. My name is Bob."
await test_agent(launch_result.agent_arn, custom_message)

## 선택사항: 로컬에서 테스트

AgentCore 런타임(Runtime)에 배포(Deployment)하기 전에 로컬에서 에이전트(Agent)를 테스트할 수 있습니다:

1. **터미널 1**: 에이전트(Agent) 시작
   ```bash
   python agent.py
   ```
   
2. **터미널 2**: 에이전트(Agent) 카드 테스트
   ```bash
   curl http://localhost:9000/.well-known/agent-card.json | jq .
   ```

3. **터미널 2**: 테스트 메시지 전송
   ```bash
   curl -X POST http://localhost:9000 \
     -H "Content-Type: application/json" \
     -d '{
       "jsonrpc": "2.0",
       "id": "req-001",
       "method": "message/send",
       "params": {
         "message": {
           "role": "user",
           "parts": [{
             "kind": "text",
             "text": "Hello! What can you do?"
           }],
           "messageId": "test-001"
         }
       }
     }' | jq .
   ```

## AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 다음을 구성(Configuration)합니다:

* 진입점: `agent.py`
* 실행 역할 자동 생성
* ECR 리포지토리 자동 생성
* 프로토콜: A2A
* Requirements 파일

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 Dockerfile이 생성됩니다.

In [ ]:
!cat client.py

## Next Steps

Now that you have successfully deployed an A2A agent with IAM authentication, you can:

1. **Add More Tools**: Extend your agent with additional tools
2. **Multi-Agent Systems**: Create orchestrator agents that call this agent
3. **Custom IAM Policies**: Create fine-grained IAM policies for agent access
4. **Integration**: Integrate with other AWS services

### Related Tutorials

* [Hosting MCP Server with IAM Auth](../02-hosting-MCP-server/hosting_mcp_server_iam_auth.ipynb)
* [Hosting A2A with JWT Auth](../05-hosting-a2a/01-a2a-getting-started-agentcore-strands.ipynb)
* [Multi-Agent Systems](../05-hosting-a2a/02-a2a-deploy-orchestrator.ipynb)

## AgentCore 런타임(Runtime)에 A2A 에이전트(Agent) 시작

이제 Dockerfile이 준비되었으니 A2A 에이전트(Agent)를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면:

1. Amazon ECR 리포지토리 생성
2. Docker 이미지 빌드 및 푸시
3. AgentCore 런타임(Runtime) 생성
4. 에이전트(Agent) 배포(Deployment)

이 작업은 몇 분이 걸릴 수 있습니다...

In [ ]:
destroy_bedrock_agentcore(
    config_path=Path(".bedrock_agentcore.yaml")
)
print("✓ AgentCore Runtime resources deleted")

## Conclusion

In this tutorial, you learned how to:

* Create an A2A agent using Strands framework
* Deploy the agent to AgentCore Runtime
* Configure IAM authentication for inbound requests
* Test the agent using SigV4-signed requests